<a href="https://colab.research.google.com/github/shin-ta/Python-study/blob/main/%E3%81%8A%E5%BC%81%E5%BD%93%E5%A3%B2%E4%B8%8A%E4%BA%88%E6%B8%AC_%E3%83%81%E3%83%A5%E3%83%BC%E3%83%88%E3%83%AA%E3%82%A2%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# グラフに日本語を表示させるためのライブラリをインストール
!pip install japanize-matplotlib

In [ ]:
# ライブラリをインポート
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import japanize_matplotlib
import seaborn as sns
sns.set(font='IPAexGothic', style='white')

In [ ]:
# SIGNATEからダウンロードしたデータをデータフレームに読み込み
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample.csv',header=None)

# データフレームのサイズを表示
print('Data Shapes')
print('Train:',train.shape, 'Test:',test.shape, 'Sample:',sample.shape)

In [ ]:
# 【参考】学習用データとテスト用データの要約情報（データ型、欠損値有無 等）を確認
print(train.info())
print('================================================')
print(test.info())

In [ ]:
# 学習用データのインデックスに[datetime]列の値を設定
train.index = pd.to_datetime(train['datetime'])
train.head()

In [ ]:
# 学習用データの統計情報を表示（数値項目）
train.describe()

In [ ]:
# 学習用データの統計情報を表示（数値項目以外）
train.describe(include='O')

In [ ]:
# 欠損値の補完と値の置き換え
train['payday'] = train['payday'].fillna(0)
train['precipitation'] = train['precipitation'].apply(lambda x : -1 if x == '--' else float(x))
train['event'] = train['event'].fillna('なし')
train['remarks'] = train['remarks'].fillna('なし')

# 月情報を保有する説明変数（特徴量）を作成
train['month'] = train['datetime'].apply(lambda x : int(x.split('-')[1]))

In [ ]:
# 弁当の売上げ（[y]列）をグラフで表示
train['y'].plot(figsize=(15,4))

In [ ]:
# 各種特徴量と弁当売上の関係を散布図で表示
fig, ax = plt.subplots(2, 3, figsize=(9,6))
train.plot.scatter(x='soldout', y='y', ax=ax[0][0])
train.plot.scatter(x='kcal', y='y', ax=ax[0][1])
train.plot.scatter(x='precipitation', y='y', ax=ax[0][2])
train.plot.scatter(x='payday', y='y', ax=ax[1][0])
train.plot.scatter(x='temperature', y='y', ax=ax[1][1])
train.plot.scatter(x='month', y='y', ax=ax[1][2])
plt.tight_layout()

In [ ]:
# 各種特徴量と弁当売上の関係を箱ひげ図で表示
fig, ax = plt.subplots(2, 2, figsize=(12,7))
sns.boxplot(x='week', y='y', data=train, ax=ax[0][0])
sns.boxplot(x='weather', y='y', data=train, ax=ax[0][1])
sns.boxplot(x='remarks', y='y', data=train, ax=ax[1][0])
ax[1][0].set_xticklabels(ax[1][0].get_xticklabels(), rotation=30)
sns.boxplot(x='event', y='y', data=train, ax=ax[1][1])
plt.tight_layout()

In [ ]:
# 「お楽しみメニュー」でなかった日の弁当の売上げ（[y]列）をグラフで表示
train[train['remarks']!='お楽しみメニュー']['y'].plot(figsize=(15,4))

# 弁当売上のグラフを重ねて表示
# train['y'].plot(c='r', ls='dashed')

In [ ]:
# 「お楽しみメニュー」かどうかの情報を保有する説明変数（特徴量）を作成
train['fun'] = train['remarks'].apply(lambda x: 1 if x=='お楽しみメニュー' else 0)

# 「お楽しみメニュー」情報と弁当売上の関係を箱ひげ図で表示
sns.boxplot(x='fun', y='y', data=train)

In [ ]:
# 「お楽しみメニュー」のデータと「お楽しみメニュー」以外のデータで中央値検定を実行
from scipy.stats import median_test
stat, p, med, tbl = median_test(train[train['fun']==1]['y'], train[train['fun']==0]['y'])
print('p', p, 'stat', stat)

In [ ]:
# 「お楽しみメニュー」の日の弁当の売上げ（[y]列）をグラフで表示
train[train['remarks']=='お楽しみメニュー']['y'].plot(figsize=(15,4))

In [ ]:
# 「お楽しみメニュー」の日のデータを表示
train[train['remarks']=='お楽しみメニュー']

In [ ]:
# メインメニューが「カレー」かどうかの情報を保有する説明変数（特徴量）を作成
train['curry'] = train['name'].apply(lambda x : 1 if x.find('カレー')>=0 else 0)

# 「カレー」情報と弁当売上の関係を箱ひげ図で表示
sns.boxplot(x='curry', y='y', data=train)

In [ ]:
# メインメニューが「カレー」のデータと「カレー」以外のデータで中央値検定を実行
stat, p, med, tbl = median_test(train[train['curry']==1]['y'], train[train['curry']==0]['y'])
print('p:', p, 'stat', stat)

モデル学習

In [ ]:
# SIGNATEからダウンロードしたデータをデータフレームに読み込み
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample.csv',header=None)

In [ ]:
# 学習用データとテスト用データを区別するフラグを付加してデータを結合する
train['t'] = 1
test['t'] = 0
dat = pd.concat([train, test],sort=True).reset_index(drop=True)

In [ ]:
# 「2014-05-01」以降のデータを取得する
dat.index = pd.to_datetime(dat['datetime'])
dat = dat['2014-05-01':]
dat = dat.reset_index(drop=True)

# 学習に必要な説明変数（特徴量）を作成
dat['days'] = dat.index
dat['precipitation'] = dat['precipitation'].apply(lambda x : -1 if x=='--' else x).astype(np.float)
dat['fun'] = dat['remarks'].apply(lambda x: 1 if x=='お楽しみメニュー' else 0)
dat['curry'] = dat['name'].apply(lambda x : 1 if x.find('カレー')>=0 else 0)

# 学習で使用する説明変数（特徴量）と目的変数をリストに格納
cols = ['precipitation', 'weather', 'days', 'fun', 'curry', 'y']

In [ ]:
# モデル学習に必要なライブラリをインポート
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as MSE
from sklearn.linear_model import LinearRegression as LR
from sklearn.ensemble import RandomForestRegressor as RF

In [ ]:
# 学習済みモデル作成関数を定義
def learning(trainX, y_train):
    # モデルのインスタンスを作成
    model1 = LR()
    model2 = RF(n_estimators=100, max_depth=4, random_state=777)

    # 経過日数を説明変数（特徴量）として、弁当売上を予測する単回帰モデルを学習
    model1.fit(trainX['days'].values.reshape(-1,1), y_train)

    # 実際の弁当売上と単回帰モデルによる売上予測の差分を特殊要因による売上として取得
    pred = model1.predict(trainX['days'].values.reshape(-1,1))
    pred_sub = y_train - pred

    # 指定した項目を説明変数（特徴量）として、特殊要因による売上を予測するモデルを学習
    model2.fit(trainX.iloc[:, ~trainX.columns.str.match('y')], pred_sub)

    # 学習済みモデルを返す
    return model1, model2

In [ ]:
# 【参考】
# ウォーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# ウォーニングを再表示する（表示設定を初期化する）
# warnings.resetwarnings()

In [ ]:
# クロスバリデーション用のデータ分割方法を定義
kf = KFold(n_splits=5, random_state=777, shuffle=True)

# クロスバリデーションの対象データを取得
tr = dat[dat['t']==1][cols]

# クロスバリデーションによる評価結果を格納するリストを初期化
trains = []
tests = []

# クロスバリデーションを実行
for train_index, test_index in kf.split(tr):
    # 訓練用データと検証用データを区別するためのフラグを設定
    tr.loc[train_index,'tt'] = 1
    tr.loc[test_index,'tt'] = 0
    tr['tt'] = tr['tt'].astype(np.int)
    tmp = pd.get_dummies(tr)
    
    # 訓練用データを取得
    trainX = tmp[tmp['tt']==1]
    del trainX['tt']

    # 検証用データを取得
    testX = tmp[tmp['tt']==0]
    del testX['tt']

    # 訓練用データ、検証用データより目的変数を取得
    y_train = tmp[tmp['tt']==1]['y']
    y_test = tmp[tmp['tt']==0]['y']
    
    # 学習済みモデル作成関数を実行
    model1, model2 = learning(trainX, y_train)
    
    # 学習済みモデルで訓練用データ、検証用データの弁当売上を予測
    pred_train = model1.predict(trainX['days'].values.reshape(-1,1)) + model2.predict(trainX.iloc[:, ~trainX.columns.str.match('y')])
    pred_test = model1.predict(testX['days'].values.reshape(-1,1)) + model2.predict(testX.iloc[:, ~testX.columns.str.match('y')])
    
    # 予測結果の評価をリストに格納
    print('TRAIN:',MSE(y_train,pred_train)**0.5, 'VARIDATE',MSE(y_test, pred_test)**0.5)
    trains.append(MSE(y_train,pred_train)**0.5)
    tests.append(MSE(y_test, pred_test)**0.5)

# 評価の平均値を表示
print('AVG')
print(np.array(trains).mean(), np.array(tests).mean())

予測実行

In [ ]:
# 結合した学習用データ、テスト用データから必要な項目を取得する
cols = ["precipitation","weather","days","fun","curry","y","t"]
tmp = pd.get_dummies(dat[cols])

# 取得したデータを学習用データとテスト用データに分割する
trainX = tmp[tmp["t"]==1]
del trainX["t"]
testX = tmp[tmp["t"]==0]
del testX["t"]

# 目的変数を作成
y_train = tmp[tmp["t"]==1]["y"]
y_test = tmp[tmp["t"]==0]["y"]

In [ ]:
# 学習用データで予測実行
model1, model2 = learning(trainX, y_train)
pred = model1.predict(trainX["days"].values.reshape(-1,1)) + model2.predict(trainX.iloc[:, ~trainX.columns.str.match("y")])

# 実際の弁当売上と予測結果をグラフで表示
p = pd.DataFrame({"actual":y_train, "pred":pred})
p.plot(figsize=(15,4))

# 予測結果の評価を表示
print("RMSE",MSE(y_train, pred)**0.5)

In [ ]:
# テスト用データで予測実行
model1, model2 = learning(trainX, y_train)
pred = model1.predict(testX["days"].values.reshape(-1,1)) + model2.predict(testX.iloc[:, ~testX.columns.str.match("y")])

# 予測結果をグラフで表示
plt.figure(figsize=(15,4))
plt.plot(pred)

In [ ]:
# テストデータの予測結果を投稿用データに設定してファイル出力
sample[1] = pred
sample.to_csv("submit0615_01.csv",index=None,header=None)